# Дескрипторы

Механизм свойств в Python позволяет контролировать доступ, изменение и удаление атрибута.

Очень безопасный класс запрещает неотрицательные значения для атрибута х:

In [2]:
class VerySafe:
    def _get_attr(self):
        return self._x
    
    def _set_attr(self, x):
        assert x > 0, "non-negative value required"
        self._x = x
    
    def _del_attr(self):
        del self._x
        
    x = property(_get_attr, _set_attr, _del_attr)

In [3]:
vary_safe = VerySafe()

In [4]:
vary_safe.x = 42

In [5]:
vary_safe.x

42

In [6]:
vary_safe.x = -42

AssertionError: non-negative value required

А если мы захотим добавить аналогичные свойства для атрибута y? Копировать? А если еще для одного атрибута?

Дескриптор - это:
+ экземпляр класса, реализующий протокол дескрипторов,
+ свойство, которое можно переиспользовать

Пример: дескриптор *NonNegative*, который делает тоже самое, что и написанные нами ранее свойства.

In [19]:
class NonNegative:
    def __get__(self, instance, owner):
        return magically_get_value(...)
    
    def __set__(self, instance, value):
        assert value >= 0, "non-negative value required"
        magically_set_value(...)
        
    def __delete__(self, instance):
        magically_delete_value(...)

In [20]:
# Проверяем

class VerySafe:
    x = NonNegative()
    y = NonNegative()

In [21]:
very_safe = VerySafe()

In [22]:
very_safe.x = 42

NameError: name 'magically_set_value' is not defined

## Протокол дескрипторов: __get__

+ Метод \_\_get__ вызывается при доступе к атрибуту.
+ Метод принимает два аргумента:
    + instance - экземпляр класса или **None**, если дескриптор был вызван в результате обращения к атрибуту у класса
    + owner - класс, "владеющий" дескриптором.
    
+ Пример:

In [23]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)
        
class A:
    attr = Descr()

In [24]:
A.attr

None <class '__main__.A'>


In [25]:
A().attr

<__main__.A object at 0x000001CAE49D0198> <class '__main__.A'>


In [26]:
# Наследники класса А будут наследовать и дескриптор

class B(A):
    pass

In [27]:
A.attr

None <class '__main__.A'>


In [28]:
A().attr

<__main__.A object at 0x000001CAE49D0518> <class '__main__.A'>


In [29]:
B.attr

None <class '__main__.B'>


## Протокол дескрипторов: __set__

+ Метод \_\_set__ вызывается для изменения значения атрибута.
+ Метод принимает два аргумента:
    + instance - экземпляр класса, "владеющего" дескриптором.
    + owner - новое значение атрибута.
    
+ Пример:

In [30]:
class Descr:
    def __set__(self, instance, value):
        print(instance, value)
        
class A:
    attr = Descr()

In [31]:
instance = A()
instance.attr = 42

<__main__.A object at 0x000001CAE49D08D0> 42


In [33]:
A.attr = 42 # Тут мы перезаписали дескриптор

## Протокол дескрипторов: __delete__

+ Метод \_\_delete__ вызывается для удаления атрибута.
+ Метод принимает один аргумент:
    + instance - экземпляр класса, "владеющего" дескриптором.
    
+ Пример:

In [34]:
class Descr:
    def __delete__(self, instance):
        print(instance)
        
class A:
    attr = Descr()

In [35]:
del A().attr

Таким образом, наши свойства должны выглядеть следующим образом:

In [44]:
class NonNegative:
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        assert value >= 0, "non-negative value required"
        self.value = value
        
    def __delete__(self, instance=None):
        del self.value

In [45]:
# Проверяем
class VerySafe:
    x = NonNegative()
    y = NonNegative()

In [46]:
very_safe = VerySafe()
very_safe.x = 42

In [47]:
very_safe.x = -42

AssertionError: non-negative value required

Все дескрипторы можно поделить на две группы:
+ дескрипторы данных aka data descriptors, определяющие как минимум метод \_\_set__
+ остальные aka non-data desctiptors

Дескриптор может быть реализован через декоратор:

In [ ]:
class property:
    def __init__(self, get=None, set=None, delete=None):
        self._get = get
        self._set = set
        self._delete = delete
        
    def __get__(self, instance, owner):
        id self._get is None:
            raise AttributeError("unreadable attribute")
        return self._get(instance)
    
    # __set__ и __delete аналогично
    
class Something:
    @property
    def attr(self):
        return 42 

Декоратор *staticmethod* позволяет объявить статический метод, т.е. просто функцию, внутри класса

In [48]:
class SomeClass:
    @staticmethod
    def do_something():
        pass

In [49]:
SomeClass.do_something()

Для объявления методов класса используется декоратор *classmethod*. Первый аргумент метода класса - непосредственно сам класс, а не его экземпляр.

In [50]:
class Settings:
    @classmethod
    def read_from(cls, path):
        return cls() # noop

In [51]:
Settings.read_from("../3.5/")

+ Как и свойства, дескрипторы позволяют контролировать
чтение, изменение и удаление атрибута, но, в отличие от
свойств, дескрипторы можно переиспользовать.
+ Дескриптор — это экземпляр класса, реализующего любую
комбинацию методов \_\_get__, \_\_set__ и \_\_delete__.

# Метаклассы

Мета класс - это класс класса.

Все классы в Python - это экземпляры класса **type**.

In [52]:
class Something:
    attr = 42

In [53]:
Something

__main__.Something

In [54]:
type(Something)

type

# Модуль abc 

Модуль abc содержит метакласс ABCMeta, который позволяет объявлять абстрактные базовые классы aka ABC.

Класс считается абстрактным, если:
+ его метакласс — ABCMeta,
+ хотя бы один из абстрактных методов не имеет конкретной реализации.


In [55]:
import abc
class Iterable(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def __iter__(self):
        pass

class Something(Iterable):
    pass

Something()

TypeError: Can't instantiate abstract class Something with abstract methods __iter__

In [60]:
from collections import deque

class MemorizingDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._history = deque(maxlen=10)
        
    def __setitem__(self, key, value):
        self._history.append(key)
        super().__setitem__(key, value)

    def get_history(self):
        return self._history

In [61]:
d = MemorizingDict({"foo": 42})
d.setdefault("bar", 24)

24

In [62]:
d["baz"] = 100500
print(d.get_history())

deque(['baz'], maxlen=10)


В CPython list, dict, set и др. — это структуры языка C, ккоторым можно обращаться через конструкции языка Python.
+ Они не предполагают расширение: dict — это не
какой-нибудь словарь, описанный в терминах __getitem__,
__setitem__ и др., а вполне конкретная его реализация.
+ Поэтому для dict перегруженный метод
MemorizingDict.__setitem__ не существует:
+ он не будет вызван в конструкторе при инициализации
словаря,

In [63]:
d = MemorizingDict({"foo": 42})

• и в методе setdefault.

In [64]:
d.setdefault("bar", 24)

24

+ Модуль collections.abc содержит абстрактные базовые
классы для коллекций на все случаи жизни.
+ Например, чтобы реализовать MemorizingDict, нужно
унаследовать его от MutableMapping и реализовать пять
методов:
     + \_\_getitem__, \_\_setitem__, \_\_delitem__,
     + \_\_iter__ и
     + \_\_len__.
+ MutableMapping выражает все остальные методы dict в
терминах этих пяти абстрактных методов.

Все встроенные коллекции являются наследниками ABC из
модуля collections.abc:

In [65]:
from collections import abc
issubclass(list, abc.Sequence)

True

In [66]:
isinstance({}, abc.Hashable)

False

• Это позволяет компактно проверять наличие у экземпляра
необходимых методов:

In [67]:
def flatten(obj):
    for item in obj:
        if isinstance(item, abc.Iterable): # ?
            yield from flatten(item)
        else:
            yield item

list(flatten([[1, 2], 3, [], [4]]))

[1, 2, 3, 4]

• Модуль abc позволяет классам в Python объявлять
абстрактные методы.